## Step 1: Import Libraries

In [5]:
# This is a Python blockimport os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

2023-07-25 06:05:56.007003: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-25 06:05:56.177042: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-25 06:05:56.177083: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-25 06:05:56.223237: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-07-25 06:05:58.697625: W tensorflow/stream_executor/pla

## Step 2: Prepare the Dataset

You'll need a dataset containing images labeled with "eyeglasses" and "no eyeglasses." Make sure you have the dataset organized into separate folders for each class.

In [1]:
!ls /work/eyeglasses-detection-notebook/deepnote/test/dataset

with_eyeglasses  without_eyeglasses


In [14]:
preprocess_data_dir = '/work/eyeglasses-detection-notebook/deepnote/test/preprocess_data'

In [11]:
dataset_dir = '/work/eyeglasses-detection-notebook/deepnote/test/dataset'

In [13]:
print(dataset_dir)
print(preprocess_data_dir)

/work/eyeglasses-detection-notebook/deepnote/test/dataset
/work/eyeglasses-detection-notebook/deepnote/test


## Step 3: Load and Preprocess Data

In [6]:
# Set the path to your dataset directory
data_dir = dataset_dir

# Define image size and batch size
img_width, img_height = 150, 150
batch_size = 32

# Use ImageDataGenerator for data augmentation and normalization
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load and preprocess training data
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

# Load and preprocess validation data
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 240 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


## Step 3: Load and Preprocess Data, Export files

In [16]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Load and preprocess training data
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
    shuffle=False  # Set shuffle to False to maintain order
)

# Load and preprocess validation data
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation',
    shuffle=False  # Set shuffle to False to maintain order
)

# Extract data and labels from the generators
train_data = np.concatenate([train_generator.next()[0] for i in range(len(train_generator))])
train_labels = np.concatenate([train_generator.next()[1] for i in range(len(train_generator))])
val_data = np.concatenate([validation_generator.next()[0] for i in range(len(validation_generator))])
val_labels = np.concatenate([validation_generator.next()[1] for i in range(len(validation_generator))])

# Save the preprocessed data and labels to files
np.save(preprocess_data_dir+'/train_data.npy', train_data)
np.save(preprocess_data_dir+'/train_labels.npy', train_labels)
np.save(preprocess_data_dir+'/val_data.npy', val_data)
np.save(preprocess_data_dir+'/val_labels.npy', val_labels)


Found 240 images belonging to 2 classes.
Found 60 images belonging to 2 classes.


## Step 4: Build the CNN Model

In [ ]:
# Create the CNN model
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()


## Step 5: Train the Model

In [ ]:
# Define the number of training and validation steps per epoch
train_steps_per_epoch = train_generator.samples // batch_size
val_steps_per_epoch = validation_generator.samples // batch_size

# Set the number of training epochs
epochs = 10

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_steps_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=val_steps_per_epoch
)

## Step 6: Evaluate the Model

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(validation_generator, steps=val_steps_per_epoch)
print(f"Validation Accuracy: {accuracy*100:.2f}%")

NameError: name 'model' is not defined

## Step 7: Make Predictions

After training and evaluating the model, you can use it to make predictions on new images. Use the model.predict() function to obtain the prediction probabilities for each class. For instance:

In [ ]:
# Assuming you have a new image (img) to predict
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)  # Add batch dimension

# Make the prediction
predictions = model.predict(img_array)
probability = predictions[0][0]

# Check if the image contains eyeglasses or not
if probability > 0.5:
    print("Eyeglasses detected!")
else:
    print("No eyeglasses detected.")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d822edbe-8d88-4e5f-b475-b8b674ae1c11' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>